In [86]:
!pip install icecream

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
!pip install -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import readers
from icecream import ic
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./settings.py
./readers.py
./train.py
./Untitled.ipynb
./utils.py
./dataset.py
./glove.6B.50d.txt
./model.py
./runner.py
./.config/.last_survey_prompt.yaml
./.config/.last_update_check.json
./.config/active_config
./.config/.last_opt_in_prompt.yaml
./.config/gce
./.config/config_sentinel
./.config/configurations/config_default
./.config/logs/2022.06.01/13.49.51.363739.log
./.config/logs/2022.06.01/13.48.45.927171.log
./.config/logs/2022.06.01/13.49.52.118738.log
./.config/logs/2022.06.01/13.49.06.780242.log
./.config/logs/2022.06.01/13.49.32.423822.log
./.config/logs/2022.06.01/13.49.25.228288.log
./data/test.scruples-anecdotes.jsonl
./data/dev.scruples-anecdotes.jsonl
./data/test.scruples-dilemmas.jsonl
./data/train.scruples-dilemmas.jsonl
./data/train.scruples-anecdotes.jsonl
./data/dev.scruples-dilemmas.jsonl
./__pycache__/settings.cpython-37.pyc
./__pycache__/readers.cpython-37.pyc
./sample_data/README.md
./sample_data/anscombe.json
./sample_data/california_housing_test.csv
./sampl

In [90]:
data_path = './data'
# glove_path = '/content/glove.6B.50d.txt'

corpus = readers.ScruplesCorpus(data_dir=data_path)

corpus2 = readers.ScruplesCorpusDataset(
            data_dir=data_path,
            split='train',
            transform=None,
            label_transform=None,
            label_scores_transform=None)

In [91]:
data = corpus.train[1]
data.head()

# data.text = data.text.apply(lambda x: x if len(str(x).split(" ")) <= 750 else np.nan)
data_text = []
data_title = []
# count = 0
for text,title in zip(data.text,data.title):
  # print(title)
  if len(text.split(" ")) <= 750 and len(title.split(" ")) > 1:
    data_text.append(text)
    data_title.append(title)
        #  if "wanting" in title:
        #    count += 1

print(len(data_text),len(data_title))
df = pd.DataFrame(list(zip(data_text, data_title)), columns =['text', 'title']) 
# print(df['text'][3],'\n',df['title'][3])
# mask = data['text'] != np.nan
# df = data.loc[mask]
# print(data.shape, df.shape)

26421 26421


In [92]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [93]:
import re
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    text = re.sub(r'\baita\b\s+','',text)
    text = re.sub(r'\bwibta\b\s+','',text)
    # text = re.sub(r'\bwanting\b\s+','',text)
    return text

def preprocess1(text):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    text = " ".join(text)
    # text = text.split()
    # newtext = []
    # for word in text:
    #     if word not in stop_words:
    #         newtext.append(word)
    # text = " ".join(newtext)
    # text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    # text = re.sub(r'\baita\b\s+','',text)
    # text = re.sub(r'\bwibta\b\s+','',text)
    # text = re.sub(r'\bwanting\b\s+','',text)
    return text

sample = "(hello) aita for wanting hi there .man tiger caller who's that isn't it ? WALL-E"
print(preprocess1(sample))

 aita for wanting hi there  . man tiger caller who is that is not it  walle


In [94]:
# data['title'] = data['title'].apply(lambda x:preprocess(x))
# data['text'] = data['text'].apply(lambda x:preprocess(x))
unpx = data['text']
unpy = data['title']
print(data['title'][20],data['text'][20])

AITA for thinking my mom wasted my time? So where I live it’s been uncharacteristically snowy. I own I a 4 wheel drive vehicle, my mom does not. She called me this morning and asked me if I would take her to the store later because she didn’t want to drive in the weather. I said sure I’ll be there around 3-4, she said perfect. 

At almost exactly 3:00 I was free, so I texted her saying I would head over to her house. My mom is horrible at checking her phone so I honestly didn’t expect a response. While I’m driving over I call her (hands free calling don’t worry!), because she’s sometimes better at answering her phone, she does not answer. Should probably note my moms house is ~10min from mine. I get to her house at ~3:15, she’s not home. I figure she took the dog for a walk, get inside, the dog is home. 

So now I’m annoyed and I call her multiple times. She finally answers at ~3:30. She says she had a dentist appointment and decided just to go grocery shopping after. I say to her “I’m

In [95]:
df['title'] = df['title'].apply(lambda x:preprocess1(x))
df['text'] = df['text'].apply(lambda x:preprocess1(x))
print(df['title'][20],df['text'][20])

x = df['text']
y = df['title']
print(x[50],y[50],sep='\n')

aita for thinking my mom wasted my time so where i live its been uncharacteristically snowy .  i own i a 4 wheel drive vehicle my mom does not .  she called me this morning and asked me if i would take her to the store later because she didnt want to drive in the weather .  i said sure ill be there around 34 she said perfect .  at almost exactly 300 i was free so i texted her saying i would head over to her house .  my mom is horrible at checking her phone so i honestly didnt expect a response .  while im driving over i call her  because shes sometimes better at answering her phone she does not answer .  should probably note my moms house is 10min from mine .  i get to her house at 315 shes not home .  i figure she took the dog for a walk get inside the dog is home .  so now im annoyed and i call her multiple times .  she finally answers at 330 .  she says she had a dentist appointment and decided just to go grocery shopping after .  i say to her im frustrated because i feel like youve

**Torch Req**

In [96]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [97]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [98]:
def readLangs(text, summary, reverse=False):
    print("Reading lines...")
    
    # Split every line into pairs and normalize
    pairs = [[text[i],summary[i]] for i in range(len(text))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(summary)
        output_lang = Lang(text)
    else:
        input_lang = Lang(text)
        output_lang = Lang(summary)

    return input_lang, output_lang, pairs

In [99]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [100]:
input_lang, output_lang, pairs = prepareData( x, y , False)
unpinput_lang, unpoutput_lang, unppairs = prepareData( unpx, unpy , False)
# print(random.choice(unppairs))

Reading lines...
Read 26421 sentence pairs
Counting words...
Counted words:
0        backstory so i got an xbox one for christmas t...
1        i work with about six other people at might jo...
2        context there was an instagram post about unpo...
3        me and my friends spent sometime organizing a ...
4        a little background .  i am a far from rich gu...
                               ...                        
26416    so a bit of background info .  my girlfriend h...
26417    context ive been best friends with this guy fo...
26418    so me  but realised it would be inappropriate ...
26419    a little info im an early college student for ...
26420    i was on the way home on a long 2 lane bridge ...
Name: text, Length: 26421, dtype: object 47901
0                            aita for hiding my controller
1                       aita for ratting out my supervisor
2        aita for commenting that unfamiliar indie band...
3        aita for arguing for my friend to surrende

In [101]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [102]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [113]:
MAX_LENGTH = 1000

In [114]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [115]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [116]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    # print(input_tensor)
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True #if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [117]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [123]:
import matplotlib.pyplot as plt
%matplotlib inline
# plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

In [118]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    print("Training....")
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(pairs[i])
                      for i in range(len(pairs))]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        if iter% 1000 == 0:
            print(iter,"/",n_iters + 1)
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [133]:
# def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
#     with torch.no_grad():
#         input_tensor = tensorFromSentence(input_lang, sentence)
#         input_length = input_tensor.size()[0]
#         encoder_hidden = encoder.initHidden()

#         encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

#         for ei in range(input_length):
#             encoder_output, encoder_hidden = encoder(input_tensor[ei],
#                                                      encoder_hidden)
#             encoder_outputs[ei] += encoder_output[0, 0]

#         decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

#         decoder_hidden = encoder_hidden

#         decoded_words = []
#         decoder_attentions = torch.zeros(max_length, max_length)

#         for di in range(max_length):
#             decoder_output, decoder_hidden, decoder_attention = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs)
#             decoder_attentions[di] = decoder_attention.data
#             topv, topi = decoder_output.data.topk(1)
#             # topkitem = decoder_output.data.topk(5)
#             # randv, randi = random.choices(topkitem, weights=topkitem[0].tolist())[0]
#             # if randi.item() == EOS_token:
#             #     decoded_words.append('<EOS>')
#             #     break
#             # else:
#             #     decoded_words.append(output_lang.index2word[randi.item()])

#             decoder_input = topi.squeeze().detach()

#         return decoded_words , decoder_attentions[:di + 1]

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [131]:
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk import word_tokenize

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words,_ = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('BLEU: ', sentence_bleu([pair[1].split()], output_sentence.split(),smoothing_function=SmoothingFunction().method4)*100)
        print('METEOR: ', meteor_score([word_tokenize(pair[1])],word_tokenize(output_sentence))*100, '\n')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
# base_decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

# Training Loss over time
# Implement decoding algos topk topp beam search
# Graph it out
# Get some examples

trainIters(encoder1, attn_decoder1, 50000, print_every=100, plot_every=500, learning_rate=0.001)

Training....
0m 24s (- 205m 10s) (100 0%) 9.1543
0m 44s (- 185m 1s) (200 0%) 8.2256
1m 4s (- 177m 26s) (300 0%) 6.8494
1m 24s (- 175m 30s) (400 0%) 6.4458
1m 47s (- 176m 40s) (500 1%) 6.1951
2m 8s (- 175m 55s) (600 1%) 5.9249
2m 30s (- 176m 11s) (700 1%) 5.8661
2m 50s (- 174m 49s) (800 1%) 5.8025
3m 13s (- 175m 39s) (900 1%) 5.5065
1000 / 50001
3m 32s (- 173m 49s) (1000 2%) 5.4296
3m 54s (- 173m 25s) (1100 2%) 5.2322
4m 15s (- 172m 56s) (1200 2%) 5.3568
4m 35s (- 171m 56s) (1300 2%) 5.2457
4m 56s (- 171m 40s) (1400 2%) 5.2433
5m 18s (- 171m 41s) (1500 3%) 5.1281
5m 38s (- 170m 26s) (1600 3%) 5.3532
6m 1s (- 171m 0s) (1700 3%) 5.1389
6m 24s (- 171m 23s) (1800 3%) 5.2430
6m 43s (- 170m 25s) (1900 3%) 5.0204
2000 / 50001
7m 4s (- 169m 59s) (2000 4%) 5.0587
7m 28s (- 170m 23s) (2100 4%) 5.2330
7m 47s (- 169m 25s) (2200 4%) 5.0247
8m 7s (- 168m 33s) (2300 4%) 5.0291
8m 29s (- 168m 19s) (2400 4%) 4.8666
8m 49s (- 167m 41s) (2500 5%) 4.9679
9m 10s (- 167m 20s) (2600 5%) 5.0352
9m 32s (- 167m 

In [122]:
plt.plot([1,2,3,4])
plt.show()

In [135]:
torch.save(encoder1.state_dict(), './enc_restriced_preprocess.w')
torch.save(attn_decoder1.state_dict(), './att_restriced_preprocess.w')

In [136]:
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder2 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

encoder2.load_state_dict(torch.load('./enc_restriced_preprocess.w'))
encoder2.eval()

attn_decoder2.load_state_dict(torch.load('./att_restriced_preprocess.w'))
attn_decoder2.eval()

AttnDecoderRNN(
  (embedding): Embedding(11535, 256)
  (attn): Linear(in_features=512, out_features=1000, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=11535, bias=True)
)

In [ ]:
evaluateRandomly(encoder2, attn_decoder2)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 58.4 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
test_text = "So I (26F) am expecting my first daughter with my husband (29M) in September. I grew up in a moderately religious household with pretty strictly defined gender roles. As in, since I was a girl, I had chores around the house while my younger brother did not have to do anything. I don't talk to my parents for a variety of reasons (going on year 5 now) so they do not know I am pregnant. Both my aunts and grandparents were very supportive of me leaving my household and as such have been my main family. Being 6 months pregnant now, people have started to give gifts and I am very grateful for them. We have decided that we don't want our daughter to be surrounded by pink everything with only girl things. So we have made a conscious effort to include traditionally 'boy' outfits and toys like dinosaurs and cars. We have told people that any gender items would be accepted and if it's getting too much 'pink' we have gone out to balance a bit (getting a green jacket or something). My younger aunt has taken great offense to this because 'boys are boys and girls are girls'. And she is a very verbal and loud person and has let us know several times her opinions. She has two boys. At my baby shower my older aunt gifted us a book called 'goodnight construction site' and a little stuffed truck. My younger aunt went on a tirade about how we are somehow making our daughter trans and not allowing her to be a girl. And she's a loud person so of course her saying this was more like a yell, and in front of my VERY liberal in-laws who are very supportive of our choice. Pregnancy hormones took over and I grabbed the toy, gave it to my younger aunt and asked her what part of the penis was needed to operate this toy truck. She yelled that's not what she meant and left. The rest of the baby shower went fine but it was a bit awkward after. Since then she has been blowing up Facebook where a bunch of people agree with her that I am making my daughter trans and have been telling me that I was an asshole. The rest of my think what we are doing is fine, but that I shouldn't have handled it that way. So was I an asshole? Edited to add: wow this really blew up, I was not anticipating this much response. To answer some questions: I do not care if she ends up loving pink and unicorns etc, we will get her them then. I just was trying to balance out the sheer amount of pink items we were getting from friends and family. Almost every single gift has been heavily pink or girl influenced. To the point where she would not be dressed in anything but pink if we did not go out and buy some more boy things. I also am generally not a person who argues. But I feel like a bit of an Ahole because I intentionally said this as deadpan serious as I could and with the intent to embarrass her in front of our family members present. My delivery and question did have the intended effect and she went quite red before leaving. This was also the first transphobic comment I've ever heard out of her mouth. While we all grew up quite conservative, my older aunt coming out 27 years ago really changed my family's perspective. My grandparents have changed from what they previously viewed and my aunt was always supportive of my other aunt. My younger aunt always wanted a girl, so I thought she was just trying to act out her fantasy girl with all her gifts and preferences."
test_label = "AITA for asking my aunt what part of the penis was required to operate a toy?"

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"].tolist(), max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = preprocess_function(data)

In [ ]:
tokenized_datatest = preprocess_function(data_test)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_datatest,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Using amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1


AttributeError: ignored

In [ ]:
pair = random.choice(unppairs)
print('>', pair[0])
print('=', pair[1])
inputs = tokenizer([pair], max_length=1024, return_tensors="pt")
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


> So I have very vivid dreams, and last night was about him cheating on me. I woke up and understood it was a dream and started to get ready for work.

I sat down and looked at his computer and had a gut feeling to check his messages. Tried to ignore it, but I ended up checking. 

I found a discord history between him and a female that i didnt recognize. All of his messages were deleted but hers were still there. She "idolizes" him and at one point acknowledges him deleting messages, but her response is: "I get it, I used to not care about cheating when I was 15 but now I'm against it". Something about it was odd to me, because she followed it with being overly cheery that hes talking to her.

I ended up messaging her saying that I was uncomfortable with the dialogue seen, reminded her that my husband is married with a child, and that if they weren't doing business (pertaining to his job) then she should fuck off.

I feel like the gut feeling was on point, but I know me going into his 

'I had a dream that my husband cheated on me. I looked at his messages and'

In [ ]:
test_text = preprocess(test_text)
test_label = preprocess(test_label)
with torch.no_grad():
  input_tensor = tensorFromSentence(input_lang, test_text)
  input_length = input_tensor.size()[0]
  encoder_hidden = encoder1.initHidden()

  encoder_outputs = torch.zeros(751, encoder1.hidden_size, device=device)

  for ei in range(input_length):
      encoder_output, encoder_hidden = encoder1(input_tensor[ei],
                                                encoder_hidden)
      encoder_outputs[ei] += encoder_output[0, 0]

  decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

  decoder_hidden = encoder_hidden

  decoded_words = []
  decoder_attentions = torch.zeros(751, 751)

  for di in range(751):
      decoder_output, decoder_hidden, decoder_attention = attn_decoder1(
          decoder_input, decoder_hidden, encoder_outputs)
      decoder_attentions[di] = decoder_attention.data
      topv, topi = decoder_output.data.topk(1)
      if topi.item() == EOS_token:
          decoded_words.append('<EOS>')
          break
      else:
          decoded_words.append(output_lang.index2word[topi.item()])

      decoder_input = topi.squeeze().detach()

print(decoded_words, decoder_attentions[:di + 1])

['telling', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 'boyfriend', 